## Generate random nodes

**Author**: Imad Saddik
<br/>
**Date**: 21/12/2024

---

**Table of contents**<a id='toc0_'></a>    
- [Load city graph](#toc1_1_)    
- [Generate random nodes](#toc1_2_)    
- [Save nodes in DB](#toc1_3_)    
- [Show nodes in map](#toc1_4_)    

---

## <a id='toc1_1_'></a>[Load city graph](#toc0_)

In [ ]:
import osmnx as ox

city_graph = ox.graph_from_place("Tanger, Maroc", network_type="drive")

## <a id='toc1_2_'></a>[Generate random nodes](#toc0_)

In [ ]:
import random


def generate_random_coordinates_from_network(graph, n):
    nodes = list(graph.nodes)
    random_nodes = random.sample(nodes, n)
    return [(graph.nodes[node]['y'], graph.nodes[node]['x']) for node in random_nodes]


employee_locations = generate_random_coordinates_from_network(city_graph, 500)
bus_depots = generate_random_coordinates_from_network(city_graph, 40)
company_location = (35.66521820595065, -5.66239083177731)

## <a id='toc1_3_'></a>[Save nodes in DB](#toc0_)

In [4]:
import psycopg2

connection = psycopg2.connect(
    dbname="routes",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
cursor = connection.cursor()

In [ ]:
from tqdm import tqdm

cursor.execute("""
CREATE TABLE IF NOT EXISTS nodes (
    id SERIAL PRIMARY KEY,
    latitude FLOAT,
    longitude FLOAT,
    type VARCHAR(255)
);
""")

for location in tqdm(bus_depots, desc="Inserting bus depots", total=len(bus_depots)):
    latitude, longitude = location
    type_ = "bus"
    cursor.execute(f"""
    INSERT INTO nodes (latitude, longitude, type)
    VALUES ({latitude}, {longitude}, '{type_}');
    """)

for location in tqdm(employee_locations, desc="Inserting employee locations", total=len(employee_locations)):
    latitude, longitude = location
    type_ = "employee"
    cursor.execute(f"""
    INSERT INTO nodes (latitude, longitude, type)
    VALUES ({latitude}, {longitude}, '{type_}');
    """)

latitude, longitude = company_location
type_ = "company"
cursor.execute(f"""
INSERT INTO nodes (latitude, longitude, type)
VALUES ({latitude}, {longitude}, '{type_}');
""")

connection.commit()
cursor.close()
connection.close()

Inserting employee locations:   0%|          | 0/500 [00:00<?, ?it/s]

Inserting employee locations: 100%|██████████| 500/500 [00:00<00:00, 13905.46it/s]


## <a id='toc1_4_'></a>[Show nodes in map](#toc0_)

In [3]:
import folium

center_lat, center_lon = company_location
my_map = folium.Map(location=[center_lat, center_lon], zoom_start=13)

for lat, lon in employee_locations:
    folium.CircleMarker(
        location=[lat, lon],
        radius=4,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=f"Employee: ({lat}, {lon})"
    ).add_to(my_map)

for lat, lon in bus_depots:
    folium.Marker(
        location=[lat, lon],
        icon=folium.Icon(color="green", icon="bus", prefix="fa"),
        popup=f"Bus Depot: ({lat}, {lon})"
    ).add_to(my_map)

folium.Marker(
    location=company_location,
    icon=folium.Icon(color="red", icon="briefcase", prefix="fa"),
    popup=f"Company: ({company_location[0]}, {company_location[1]})"
).add_to(my_map)

my_map.save("../html/nodes_plotted_in_map.html")

In [4]:
my_map